# Heuristics
___

In [20]:
import numpy as np

## Marouane Heuristic 

In [21]:
def johnsons_rule(machine1, machine2):
    # Sort jobs based on min(AM1_i, AM2_i)
    artificial_jobs = list(zip(machine1, machine2))

    jobs_sorted = sorted(enumerate(artificial_jobs), key=lambda x: min(x[1]))
    U = [job for job in jobs_sorted if job[1][0] < job[1][1]]
    V = [job for job in jobs_sorted if job[1][0] >= job[1][1]]
    # Concatenate U in order and V in reverse order, extracting only job indices
    sequence = [job[0] for job in U] + [job[0] for job in reversed(V)]
    return sequence

In [22]:
def calculate_makespan(processing_times, sequence):
    n_jobs = len(sequence)
    n_machines = len(processing_times[0])
    end_time = [[0] * (n_machines + 1) for _ in range(n_jobs + 1)]
    
    for j in range(1, n_jobs + 1):
        for m in range(1, n_machines + 1):
            end_time[j][m] = max(end_time[j][m - 1], end_time[j - 1]
                                 [m]) + processing_times[sequence[j - 1]][m - 1]

    return end_time[n_jobs][n_machines]

In [23]:
def marouane_heuristique(processing_times):
    n_jobs,n_machines = processing_times.shape
    best_sequence = None
    best_makespan = float('inf')

    for k in range(1,n_machines - 1):  # Generate all artificial 2-machine problems
        weights_front = np.array( [n_machines - i for i in range(k) ])
        weights_back = np.array([i + 1 for i in range(k,n_machines)])
        AM1 = processing_times[:, :k].dot(weights_front)
        AM2 = processing_times[:, k:].dot(weights_back)

        # artificial_jobs = list(zip(AM1, AM2))
        sequence = johnsons_rule(AM1 , AM2)
        makespan = calculate_makespan(processing_times, sequence)
        if makespan < best_makespan:
            best_makespan = makespan
            best_sequence = sequence

    return best_sequence

## GUPTA's Method

In [24]:
def min_sum_processing(job_index, processing_times):
    min_sum = np.inf
    for i in range(processing_times.shape[1] - 1):
        sum_for_pair = processing_times[job_index,
                                        i] + processing_times[job_index, i + 1]
        if sum_for_pair < min_sum:
            min_sum = sum_for_pair
    return min_sum



In [25]:

def calculate_priority(job_index, processing_times):
    diff = float(processing_times[job_index, 0] -
                 processing_times[job_index, -1])
    sign = (diff > 0) - (diff < 0)
    return sign / min_sum_processing(job_index, processing_times)

In [26]:


def gupta_heuristic(processing_times):
    priorities = [calculate_priority(i, processing_times)
                  for i in range(processing_times.shape[0])]
    total_times = [np.sum(processing_times[i])
                   for i in range(processing_times.shape[0])]
    return sorted(range(len(priorities)), key=lambda k: (priorities[k], total_times[k]))

In [27]:
processing_times = np.array([[15, 28, 77,  1, 45],
                                   [64,  4, 36, 59, 73],
                                   [64, 43, 57, 95, 59],
                                   [48, 93, 15, 49, 63],
                                   [9,  1, 81, 90, 54],
                                   [91, 81, 82, 78, 98],
                                   [27, 77, 98,  3, 39],
                                   [34, 69, 97, 69, 75],
                                   [42, 52, 12, 99, 33],
                                   [3, 28, 35, 41,  8],
                                   [11, 28, 84, 73, 86],
                                   [54, 77, 70, 28, 41],
                                   [27, 42, 27, 99, 41],
                                   [30, 53, 37, 13, 22],
                                   [9, 46, 59, 59, 43],
                                   [15, 49, 42, 47, 34],
                                   [88, 15, 57,  8, 80],
                                   [55, 43, 16, 92, 16],
                                   [50, 65, 11, 87, 37],
                                   [57, 41, 34, 62, 94]])

best_sequence = marouane_heuristique(processing_times)
print("Best sequence:", best_sequence)
print("Best makespan:", calculate_makespan(processing_times, best_sequence))

Best sequence: [4, 12, 14, 10, 1, 8, 19, 2, 5, 7, 18, 3, 17, 16, 15, 11, 0, 6, 9, 13]
Best makespan: 1387
